### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [250]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV


# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes')) # returns a list of the raw texts that can be fed to text feature extractor, si usamos sklearn.datasets.fetch_20newsgroups_vectorized no hace falta vectorizar, ya viene vectorizado.
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [42]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0], len(newsgroups_train.data), list(newsgroups_train.target_names)

('I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.',
 11314,
 ['alt.atheism',
  'comp.graphics',
  'comp.os.ms-windows.misc',
  'comp.sys.ibm.pc.hardware',
  'comp.sys.mac.hardware',
  'comp.windows.x',
  'misc.forsale',
  'rec.autos',
  'rec.motorcycles',
  'rec.sport.baseball',
  'rec.sport.hockey',
  'sci.crypt',
  'sci.electronics',
  'sci.med',
  'sci.space',
  'soc.religion.christian',
  'talk.politics.guns',
  'talk.politics.mideast',
  'talk.politics.misc',
  'talk.religion.misc'])

In [18]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [44]:
X_train.shape

(11314, 101631)

Tenemos 11314 documentos y 101631 palabras distintas entre los documentos, X_train es una matriz, donde cada vector-fila representa un documento y cada valor del vector representa una palabra, que tan "importante" es esa palabra en el documento especifico.

In [45]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [51]:
print(f"Promedio de componentes que no son ceros en los vectores: {X_train.nnz / float(X_train.shape[0])} de un total de {X_train.shape[1]}")

Promedio de componentes que no son ceros en los vectores: 97.54525366802191 de un total de 101631


In [20]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [52]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [53]:
# en `y_train` guardamos los targets que son enteros, es el indice de la categoria
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [54]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [55]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [57]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [58]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [60]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  6385,  1149, 11238])

In [61]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [62]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [63]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [64]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [65]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [66]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

np.float64(0.5854345727938506)

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno. Analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [112]:
newsgroups_train.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [122]:
# creamos todos los idx posibles
idxs = range(len(newsgroups_train.data))

# tomamos 5 indices al azar
five_idxs = np.random.choice(idxs, 5)

# recorremos los indices:
for id in five_idxs:
    # calculamos las similitudes
    cossim = cosine_similarity(X_train[id], X_train)[0]

    # indices de los 5 docs mas similares:
    similar_docs_idx_list = np.argsort(cossim)[::-1][1:6]

    # Etiquete de clasificación de los documentos similares:
    type_similar_document_list = [newsgroups_train.target_names[y_train[i]] for i in similar_docs_idx_list]

    # Imprimimos la calidad de las coincidencias y los indices de los docs 
    print(f"Calidad de coincidencias: {np.sort(cossim)[::-1][1:6]} e indices de los docs: {similar_docs_idx_list}")

    print(f"Documento original: {newsgroups_train.target_names[y_train[id]]}, Similares: {type_similar_document_list}")

    # imprimimos algunas lineas de cada documentos para ver si tienen algo en común
    original_document_lines = newsgroups_train.data[id].split('\n')
    some_original_document_lines = "".join(original_document_lines[:5])
    print("ABOUT ORIGINAL DOCUMENT: ", some_original_document_lines)
    for idx in similar_docs_idx_list:

        lines = newsgroups_train.data[idx].split('\n')

        some_lines = "".join(lines[:5])

        print(f"ABOUT SIMILAR DOCUMENT {idx}: ", some_lines)

    print("\n\n")

Calidad de coincidencias: [0.81589481 0.23763918 0.23197704 0.21962054 0.21844707] e indices de los docs: [  830  7879  5708 10489 11017]
Documento original: misc.forsale, Similares: ['misc.forsale', 'rec.autos', 'misc.forsale', 'misc.forsale', 'misc.forsale']
ABOUT ORIGINAL DOCUMENT:  Misc. Items for sale:Walkman:  Aiwa Model HS-T30, 1 year old, mint condition, hardly used,           autoreverse, 3 band Equalizer, Super Bass, Dolby Noise Reduction,
ABOUT SIMILAR DOCUMENT 830:  Misc. Items for sale:Mount Plate:  Sony Model CPM-203P, mounting plate for Sony portable CD playersfor Portable: plugs into car lighter, snaps onto the bottom of any Sony
ABOUT SIMILAR DOCUMENT 7879:  My friend, David Gordon wants to sell his 1989 Honda. Some of the details of the car are as follows:             Five speed             A/c, AM/FM/Cassette stereo
ABOUT SIMILAR DOCUMENT 5708:  I have a used Sony D-808K CarDiscman for sale.  I bought it new on June 16, 1992.  It still has the one-year warranty intac

Mayor valor que devuelve la coincidencia de coseno, mayor coincidencia tienen los documentos.  
Por ejemplo en el primer caso, hay altos valores devuelto por la coincidencia de coseno, y todos los documentos devueltos hablan sobre ventas.  
En el segundo caso, no hay buenos valores devueltos, y los documentos no son tan similares entre sí.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

Primer caso: MultinomialNB

In [256]:
# Instanciamos un vectorizador:
tfidfvect = TfidfVectorizer(strip_accents=None, 
                            lowercase=True, 
                            analyzer="word", 
                            ngram_range=(1, 1),
                            norm="l2",
                            sublinear_tf=False,
                            )

# Aplicamos la vectorización a los distintos datsets:
X_train= tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

# Creamos y entrenamos el modelo
clf = MultinomialNB(alpha=0.1,
                    fit_prior=False, 
                    )
clf.fit(X_train, y_train)

# Hacemos la predicciones y calculamos el F1-score
y_pred =  clf.predict(X_test)
f1_score(y_test, y_pred, average='macro')

np.float64(0.6664185318178519)

Vectorizador:  
- Caso sin tocar parametros: np.float64(0.5854345727938506).  
- Cambiar strip_accents no afecto en nada, se obtuvo lo mismo.  
- lowercase=False redujo: np.float64(0.563325189561675).   
- analyzer="char_wb" o "char" redujo: np.float64(0.10050507899086067).  
- ngram_range=(2, 2) o (1, 2) o (2, 3) redujo: np.float64(0.44465359773367946).  
- norm="l1" redujo a: np.float64(0.4949628779936094).
- sublinear_tf=True, redujo: np.float64(0.6166664953246763).

Modelo:  
- Reducir alpha mejora la metrica F1.
- cambia fit_prior=False mejora un poco.  

Logramos mejorar casi un 10% la calidad inicial del modelo.

Segundo caso: ComplementNB

In [257]:
# Instanciamos un vectorizador:
tfidfvect = TfidfVectorizer(strip_accents=None,
                            lowercase=True,
                            analyzer="word",
                            ngram_range=(1, 1),
                            norm="l2",
                            sublinear_tf=True
                            )

# Aplicamos la vectorización a los distintos datsets:
X_train= tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

# Creamos y entrenamos el modelo
clf = ComplementNB(alpha= 0.6,
                   fit_prior=True,
                   norm=False
                    )
clf.fit(X_train, y_train)

# Hacemos la predicciones y calculamos el F1-score
y_pred =  clf.predict(X_test)
f1_score(y_test, y_pred, average='macro')

np.float64(0.6947966110126378)

- Sin tocar ningún parametro: np.float64(0.692953349950875).

Vectorizador:  
- Cambiar strip_accents no afecto en nada, se obtuvo lo mismo.  
- lowercase=False redujo: np.float64(0.6892731374398724).   
- analyzer="char_wb" o "char" redujo: np.float64(0.12122769736189094).  
- ngram_range=(2, 2) o (1, 2) o (2, 3) redujo: np.float64(0.5567051871233293). 
- norm="l1" redujo a: np.float64(0.6657036625279055).
- sublinear_tf=True, mejoro: np.float64(0.6951325517946223).

Modelo:
- Reducir alpha mejoro algo: np.float64(0.6965218464867293).
- Cambiar fit_prior no tuvo cambios.
- norm=True redujo la metrica: np.float64(0.6925625135005732).

Mejoramos un 12% en comparación al caso inicial de usar el modelo MultinomialNB.

Todo lo anterior fue hecho manualmente, esto también se puede hacer con una grilla:

In [258]:
# creamos la instancia del vectorizador
tfidfvect = TfidfVectorizer(strip_accents=None,
                            lowercase=True,
                            analyzer="word",
                            ngram_range=(1, 1),
                            norm="l2",
                            sublinear_tf=True)

# Aplicamos la vectorización a los distintos datsets:
X_train= tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

# Creamos y entrenamos el modelo
clf = ComplementNB()

# creamos el mapa de grilla
grid_map = {
    'alpha': [0.1, 0.2, 0.3, 0.4, 0.7, 0.9, 1, 2, 5, 10],
    'fit_prior': [True, False],
    'norm': [True, False]
}

# creamos la grilla
grid = GridSearchCV(
    estimator=clf,
    param_grid=grid_map,
    scoring="f1_macro",
    n_jobs=-1,
    refit=True,
    cv=5,
    verbose=1
)

# aplicamos la grilla
grid.fit(X_train, y_train)

# Evaluar los resultados
print(f"Mejores parámetros: {grid.best_params_}")
print(f"Mejor score: {grid.best_score_}")

best_model = grid.best_estimator_

# Hacemos la predicciones y calculamos el F1-score
y_pred =  best_model.predict(X_test)
f1_score(y_test, y_pred, average='macro')

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Mejores parámetros: {'alpha': 0.3, 'fit_prior': True, 'norm': False}
Mejor score: 0.7667088131250549


np.float64(0.6989825595098211)

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


Hasta el momento tenemos una matriz donde las filas representan los documentos y las columnas representan las palabras. Transponer significa que las filas representaran a las palabras y las columnas a los documentos.

In [259]:
X_train_transpose = X_train.T

X_train_transpose.shape, X_train.shape

((101631, 11314), (11314, 101631))

In [298]:
# Para buscar los indices de palabras
tfidfvect.vocabulary_['authoratative']

# car --> indice 25775
# photographer --> indice 71223
# authoratative --> indice 20496
# administrators --> indice 16907
# librarian --> indice 56601


20496

In [299]:
# almacenamos las palabras en una lista
words_list = [idx2word[25775], idx2word[71223], idx2word[20496], idx2word[16907], idx2word[56601]]
idx_list = [tfidfvect.vocabulary_['car'], tfidfvect.vocabulary_['photographer'], tfidfvect.vocabulary_['authoratative'], tfidfvect.vocabulary_['administrators'], tfidfvect.vocabulary_['librarian']]

words_list, idx_list

(['car', 'photographer', 'authoratative', 'administrators', 'librarian'],
 [25775, 71223, 20496, 16907, 56601])

In [300]:
# recorremos los indices:
for id in idx_list:
    # calculamos las similitudes
    cossim = cosine_similarity(X_train_transpose[id], X_train_transpose)[0]

    # indices de las 5 palabras mas similares:
    similar_words_idx_list = np.argsort(cossim)[::-1][1:6]

    print(f"Para {idx2word[id]} las palabras similares son: ")
    for i in similar_words_idx_list:
        print(f"{idx2word[i]}")

    print("\n\n")

Para car las palabras similares son: 
cars
dealer
owner
civic
the



Para photographer las palabras similares son: 
meskhetians
azerbaijanians
rasulzade
inhumanly
westernizers



Para authoratative las palabras similares son: 
legitimacy
scintific
unscintific
woring
inself



Para administrators las palabras similares son: 
confusingly
pornographic
manifestation
broadcasting
adminstrators



Para librarian las palabras similares son: 
residual
actrix
progression
textbook
weakness



